###  Building a postgres database for crime reports

The goal of this guided project is for you to create a database named crimes_db with a table – boston_crimes – with appropriate datatypes for storing the data from the boston.csv file.

You will be creating the table inside a schema named crimes. You will also create the readonly and readwrite groups with the appropriate privileges as we have discussed in this course. Finally, you will also need to create one user for each of these groups.

This dataset is available in the file boston.csv. The first four rows are shown here:

<img src="first_five_row.png">

In [1]:
import psycopg2

# establishing the connection and create 
conn = psycopg2.connect(
   database="postgres", user='postgres', password='123456', host='127.0.0.1', port= '5432'
)
cur = conn.cursor()
conn.autocommit = True
cur.execute("DROP DATABASE crime_db")
cur.execute("CREATE DATABASE crime_db;")
conn.autocommit = False
conn.close()

In [2]:
# CONNECT TO THE crime_db
conn = psycopg2.connect(
    dbname="crime_db", user="postgres", password="123456", host='127.0.0.1', port= '5432'
)
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes")
conn.commit()
conn.close()

In [3]:
import csv
with open("boston.csv", mode='r') as file:
    reader = csv.reader(file)
    (col_headers, first_row)=[row for idx, row in enumerate(reader) if idx in (0,1)]
    print(col_headers, first_row)

['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long'] ['1', '619', 'LARCENY ALL OTHERS', '2018-09-02', 'Sunday', '42.35779134', '-71.13937053']


In [4]:
# identify the proper datatype for the columns
def get_col_set(csv_filename, col_index):
    with open(csv_filename) as file:
        reader = csv.reader(file)
        next(reader)
        col_result = set()
        for row in reader:
            col_result.add(row[col_index])
    return col_result

for index in range(len(col_headers)):
    result = get_col_set('boston.csv',index)
    print('The first 10 elements in Column {}'.format(index))
    print('The length of Column {} is {}'.format(index, len(result)))
    column = list(result)
    print(column[:10])
    print('\n')

The first 10 elements in Column 0
The length of Column 0 is 298329
['244802', '265135', '106122', '213500', '233918', '10756', '55104', '95131', '276256', '221']


The first 10 elements in Column 1
The length of Column 1 is 219
['2617', '1843', '613', '3830', '3625', '3029', '623', '3203', '1842', '1601']


The first 10 elements in Column 2
The length of Column 2 is 239
['DISORDERLY CONDUCT', 'PROSTITUTION', 'ROBBERY - STREET', 'PROPERTY - RECEIVING STOLEN', 'M/V ACCIDENT - PROPERTY Â\xa0DAMAGE', 'LARCENY THEFT OF BICYCLE', 'M/V ACCIDENT - INVOLVING PEDESTRIAN - NO INJURY', 'ANNOYING AND ACCOSTIN', 'CRIMINAL HARASSMENT', 'ASSAULT - AGGRAVATED']


The first 10 elements in Column 3
The length of Column 3 is 1177
['2017-03-19', '2016-07-13', '2018-02-17', '2017-03-07', '2018-02-15', '2016-07-11', '2015-10-08', '2017-02-06', '2018-04-28', '2015-11-02']


The first 10 elements in Column 4
The length of Column 4 is 7
['Friday', 'Thursday', 'Tuesday', 'Sunday', 'Monday', 'Wednesday', 'Saturda

In [5]:
# Column incident_number INTEGER
# Column offense_code smallint
# Column description varchar(n)
# Column date timestamp
# Column day_of_the_week ENUM
# Column lat float8
# Column long float8

# Find out the maximum number of digit in description text
column_desc = list(get_col_set('boston.csv', 2))
def find_max_number_char(column_set_as_list):
    max = 0
    for item in column_set_as_list:
        if len(item) > max:
            max = len(item)
    return max
print(find_max_number_char(column_desc))

58


In [6]:
conn = psycopg2.connect(
    dbname="crime_db", user="postgres", password="123456", host='127.0.0.1', port= '5432'
)
cur = conn.cursor()

cur.execute(
    """
    CREATE TYPE week_of_the_day_enum AS ENUM (
    'Sunday', 'Monday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Tuesday'
    );
    """
)

cur.execute(
    """
    CREATE TABLE crimes.boston_crimes(
        incident_number INTEGER PRIMARY KEY,
        offense_code smallint,
        description varchar(60),
        date timestamp,
        week_of_the_day week_of_the_day_enum,
        lat float8,
        long float8
    );
    """
)

conn.commit()

In [7]:
conn = psycopg2.connect(
    dbname="crime_db", user="postgres", password="123456", host='127.0.0.1', port= '5432'
)
cur = conn.cursor()
with open('boston.csv', mode='r') as file:
    cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER", file)
conn.commit()

In [8]:
conn = psycopg2.connect(
    dbname="crime_db", user="postgres", password="123456", host='127.0.0.1', port= '5432'
)
cur = conn.cursor()
cur.execute("""
SELECT 
   *
FROM 
   crimes.boston_crimes
LIMIT 10
""")
result = cur.fetchmany(10)
print(result)
conn.close()

[(1, 619, 'LARCENY ALL OTHERS', datetime.datetime(2018, 9, 2, 0, 0), 'Sunday', 42.35779134, -71.13937053), (2, 1402, 'VANDALISM', datetime.datetime(2018, 8, 21, 0, 0), 'Tuesday', 42.30682138, -71.06030035), (3, 3410, 'TOWED MOTOR VEHICLE', datetime.datetime(2018, 9, 3, 0, 0), 'Monday', 42.34658879, -71.07242943), (4, 3114, 'INVESTIGATE PROPERTY', datetime.datetime(2018, 9, 3, 0, 0), 'Monday', 42.33418175, -71.07866441), (5, 3114, 'INVESTIGATE PROPERTY', datetime.datetime(2018, 9, 3, 0, 0), 'Monday', 42.27536542, -71.09036101), (6, 3820, 'M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY', datetime.datetime(2018, 9, 3, 0, 0), 'Monday', 42.29019621, -71.07159012), (7, 724, 'AUTO THEFT', datetime.datetime(2018, 9, 3, 0, 0), 'Monday', 42.30607218, -71.0827326), (8, 3301, 'VERBAL DISPUTE', datetime.datetime(2018, 9, 3, 0, 0), 'Monday', 42.32701648, -71.10555088), (9, 301, 'ROBBERY - STREET', datetime.datetime(2018, 9, 3, 0, 0), 'Monday', 42.33152148, -71.07085307), (10, 3301, 'VERBAL DISPUTE', dat

Now it is time to handle users. Our goal is to create the two user groups: readonly and readwrite. By following the [least privilege principle](https://en.wikipedia.org/wiki/Principle_of_least_privilege), the first step in doing so is to make sure that there are no privileges inherited from the public group and on the public schema.

In [9]:
conn = psycopg2.connect(
    dbname="crime_db", user="postgres", password="123456", host='127.0.0.1', port= '5432'
)
cur = conn.cursor()
cur.execute("REVOKE ALL ON DATABASE crime_db FROM public;")
cur.execute("REVOKE ALL ON SCHEMA public FROM public;")
conn.commit()

In [10]:
conn = psycopg2.connect(
    dbname="crime_db", user="postgres", password="123456", host='127.0.0.1', port= '5432'
)
cur = conn.cursor()
cur.execute("DROP GROUP readonly")
cur.execute("CREATE GROUP readonly NOLOGIN")
cur.execute("REVOKE ALL ON crimes.boston_crimes from readonly;")
cur.execute("GRANT SELECT ON crimes.boston_crimes TO readonly;")

cur.execute("DROP GROUP readwrite")
cur.execute("CREATE GROUP readwrite NOLOGIN")
cur.execute("REVOKE ALL ON crimes.boston_crimes from readwrite")
cur.execute("GRANT SELECT, INSERT, UPDATE, DELETE ON crimes.boston_crimes TO readwrite;")

conn.commit()

In [11]:
conn = psycopg2.connect(
    dbname="crime_db", user="postgres", password="123456", host='127.0.0.1', port= '5432'
)
cur = conn.cursor()
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readonly")
cur.execute("GRANT CONNECT ON DATABASE crime_db TO readwrite")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readonly")
cur.execute("GRANT USAGE ON SCHEMA crimes TO readwrite")
conn.commit()

In [12]:
conn = psycopg2.connect(
    dbname="crime_db", user="postgres", password="123456", host='127.0.0.1', port= '5432'
)
cur = conn.cursor()
cur.execute("DROP USER data_analyst")
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1'")
cur.execute("GRANT readonly TO data_analyst")
cur.execute("DROP USER data_scientist")
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2'")
cur.execute("GRANT readwrite TO data_scientist")
conn.commit()

In [13]:
conn = psycopg2.connect(
    dbname="crime_db", user="postgres", password="123456", host='127.0.0.1', port= '5432'
)
cur = conn.cursor()
cur.execute("SELECT * FROM pg_user")
print(cur.fetchall())

cur.execute("""
    SELECT grantee, privilege_type
        FROM information_schema.table_privileges
        WHERE grantee = 'readwrite';
""")
print(cur.fetchall())

cur.execute("""
    SELECT grantee, privilege_type
        FROM information_schema.table_privileges
        WHERE grantee = 'readonly';
""")
print(cur.fetchall())

[('postgres', 10, True, True, True, True, '********', None, None), ('data_analyst', 49328, False, False, False, False, '********', None, None), ('data_scientist', 49329, False, False, False, False, '********', None, None)]
[('readwrite', 'INSERT'), ('readwrite', 'SELECT'), ('readwrite', 'UPDATE'), ('readwrite', 'DELETE')]
[('readonly', 'SELECT')]


<img src="crime_db_relation.png">